In [107]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import pymongo

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [102]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

story = soup.find('div', class_='list_text')
title = story.find('div', class_='content_title').text
dek = story.find('div', class_='article_teaser_body').text
print(story)

<div class="list_text"><div class="list_date">October  7, 2019</div><div class="content_title"><a href="/news/8522/nasas-curiosity-rover-finds-an-ancient-oasis-on-mars/" target="_self">NASA's Curiosity Rover Finds an Ancient Oasis on Mars</a></div><div class="article_teaser_body">New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.</div></div>


In [103]:
fimg_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(fimg_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
footers = soup.find_all('footer')
fimg = footers[0].find('a')
featured_image_url = 'https://www.jpl.nasa.gov' + fimg.get('data-fancybox-href')
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23170_ip.jpg'

In [104]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_weather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
print(mars_weather.text)

InSight sol 313 (2019-10-14) low -101.4ºC (-150.6ºF) high -27.2ºC (-16.9ºF)
winds from the SSE at 5.5 m/s (12.4 mph) gusting to 22.5 m/s (50.3 mph)
pressure at 7.20 hPapic.twitter.com/2moNAouxXa


In [105]:
fact_url = 'https://space-facts.com/mars/'
browser.visit(fact_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
facts = soup.find(id='tablepress-comp-mars')
df = pd.read_html(str(facts))[0].rename(columns={'Mars - Earth Comparison': 'Metric', 
                                             'Mars': 'Mars', 'Earth': 'Earth'})
df = df.set_index('Metric').drop('Earth', axis = 1)
mars_table = df.to_html()

In [106]:
mars_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Metric</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n  </tbody>\n</table>'

In [144]:
mars_table[40:-10].replace('\n', '')

'thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>    </tr>    <tr>      <th>Metric</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-153 to 20 °C</td>    </tr>  </tbody'

In [130]:
df

,Mars
Metric,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-153 to 20 °C


In [108]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_db

In [115]:
db.mars.drop()

In [117]:
db.mars.insert_one({'title': 'blah', 'weather': 'bldkha;khga;ekhf;aek'})

In [119]:
elems = list(db.mars.find())

In [123]:
elems[0]['title']

'blah'

In [86]:
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
hemispheres = []
items = soup.find_all('div', class_='item')
hem_names = ['Cerberus', 'Valles Marineris', 'Schiaparelli', 'Syrtis Major']
for hem, item in zip(hem_names, items):
    browser.click_link_by_partial_href(hem.lower())
    himg = item.find('a', class_='target')
    hemispheres.append({'title': f'{hem} Hemisphere', 'img_url': himg.get('href')})

ElementClickInterceptedException: Message: element click intercepted: Element <a href="/search/map/Mars/Viking/cerberus_enhanced" class="itemLink product-item">...</a> is not clickable at point (22, 204). Other element would receive the click: <div class="item">...</div>
  (Session info: chrome=77.0.3865.120)
